# CNN - ConvNextV2

https://huggingface.co/facebook/convnextv2-tiny-1k-224

In [2]:
import pandas as pd
from transformers import pipeline

classifier = pipeline("image-classification", model="facebook/convnextv2-tiny-1k-224")

t:\win_programs\python_venvs\gnn-homeworks\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from PIL import Image
import os
import zipfile
from tqdm import tqdm
import shutil

    
def classify_images(zip_name, output_suffix, drive_path_root):

    path_to_zip = drive_path_root + zip_name
    path_extracted = "/content" + zip_name

    print('ZIP source:', path_to_zip)
    print("Extracting to", path_extracted)

    os.makedirs(path_extracted, exist_ok=True)

    with zipfile.ZipFile(path_to_zip, 'r') as zip_ref:
        zip_ref.extractall(path_extracted)

    image_paths = [path_extracted +'/'+p for p in os.listdir(path_extracted) if p.endswith(".png")]
    print(f'Found {len(image_paths)} images in {path_extracted}')

    model_predictions_top_5 = []
    model_predictions_top_1 = []

    for image_path in tqdm(image_paths):
        image = Image.open(image_path).convert("RGB")
        prediction = classifier(image)
        model_predictions_top_5.append({
            "image_path": image_path,
            "predictions": prediction
        })
        model_predictions_top_1.append({
            "image_path": image_path,
            "prediction": prediction[0]['label'],
            "score": round(prediction[0]['score'], 2)
        })

    output_top_1_name = f"classifier_predictions_top_1_{output_suffix}.csv"
    output_top_5_name = f"classifier_predictions_top_5_{output_suffix}.csv"

    # save model_predictions_top_1 as a csv
    df = pd.DataFrame(model_predictions_top_1)
    df.to_csv(output_top_1_name, index=True)

    # save model_predictions_top_5 as a csv
    # model_predictions_top_5 has a nested list of dictionaries, so we need to flatten it
    model_predictions_all_flat = []
    for image in model_predictions_top_5:
        for prediction_position, prediction in enumerate(image['predictions']):
            model_predictions_all_flat.append({
                "image_path": image['image_path'],
                "prediction_position": prediction_position,
                "label": prediction['label'],
                "score": round(prediction['score'], 2)
            })

    # save model_predictions_all_flat as a csv
    df = pd.DataFrame(model_predictions_all_flat)
    df.to_csv(output_top_5_name, index=True)

    # copy to google drive
    shutil.copyfile(output_top_1_name, drive_path_root + '/' + zip_name.split('/')[1] + '/' + output_top_1_name)
    print("Finished, saved to", drive_path_root + '/' + zip_name.split('/')[1] + '/' + output_top_1_name)

    shutil.copyfile(output_top_5_name, drive_path_root + '/' + zip_name.split('/')[1] + '/' + output_top_5_name)
    print("Finished, saved to", drive_path_root + '/' + zip_name.split('/')[1] + '/' + output_top_5_name)

    # clean up
    shutil.rmtree(path_extracted)

In [40]:
# root_path = "datasets_generated_ready_models"
# zip_name = "/SD_XL/cars_zipped.zip"
# output_prefix = 'cars'

# classify_images(zip_name, output_suffix, drive_path_root=root_path)

In [42]:
root_path = "datasets_generated_ready_models"
zip_names_and_suffices = [
    ("/DALLE3/dalle_cars_zipped.zip", "cars"),
    ("/DALLE3/dalle_wildlife_zipped.zip", "wildlife"),

    ("/SD_XL/cars_zipped.zip", "cars"),
    ("/SD_XL/wildlife_zipped.zip", "wildlife"),
]

for zip_name, output_suffix in zip_names_and_suffices:
    classify_images(zip_name, output_suffix, drive_path_root=root_path)

ZIP source: datasets_generated_ready_models/DALLE3/dalle_cars_zipped.zip
Extracting to /content/DALLE3/dalle_cars_zipped.zip
Found 50 images in /content/DALLE3/dalle_cars_zipped.zip


100%|██████████| 50/50 [00:12<00:00,  4.13it/s]


Finished, saved to datasets_generated_ready_models/DALLE3/classifier_predictions_top_1_cars.csv
Finished, saved to datasets_generated_ready_models/DALLE3/classifier_predictions_top_5_cars.csv
ZIP source: datasets_generated_ready_models/DALLE3/dalle_wildlife_zipped.zip
Extracting to /content/DALLE3/dalle_wildlife_zipped.zip
Found 50 images in /content/DALLE3/dalle_wildlife_zipped.zip


100%|██████████| 50/50 [00:17<00:00,  2.91it/s]


Finished, saved to datasets_generated_ready_models/DALLE3/classifier_predictions_top_1_wildlife.csv
Finished, saved to datasets_generated_ready_models/DALLE3/classifier_predictions_top_5_wildlife.csv
ZIP source: datasets_generated_ready_models/SD_XL/cars_zipped.zip
Extracting to /content/SD_XL/cars_zipped.zip
Found 50 images in /content/SD_XL/cars_zipped.zip


100%|██████████| 50/50 [00:16<00:00,  3.09it/s]


Finished, saved to datasets_generated_ready_models/SD_XL/classifier_predictions_top_1_cars.csv
Finished, saved to datasets_generated_ready_models/SD_XL/classifier_predictions_top_5_cars.csv
ZIP source: datasets_generated_ready_models/SD_XL/wildlife_zipped.zip
Extracting to /content/SD_XL/wildlife_zipped.zip
Found 50 images in /content/SD_XL/wildlife_zipped.zip


100%|██████████| 50/50 [00:15<00:00,  3.27it/s]

Finished, saved to datasets_generated_ready_models/SD_XL/classifier_predictions_top_1_wildlife.csv
Finished, saved to datasets_generated_ready_models/SD_XL/classifier_predictions_top_5_wildlife.csv


# Object detection

Source: https://docs.ultralytics.com/models/yolov8/

In [29]:
from ultralyticsplus import YOLO

# load model
model = YOLO('ultralyticsplus/yolov8m')

# known classes from https://huggingface.co/ultralyticsplus/yolov8s
known_classes = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

# set model parameters
model.overrides['conf'] = 0.25  # NMS confidence threshold
model.overrides['iou'] = 0.45  # NMS IoU threshold
model.overrides['agnostic_nms'] = False  # NMS class-agnostic
model.overrides['max_det'] = 100  # maximum number of detections per image

In [26]:
from PIL import Image
import os
import zipfile
from tqdm import tqdm
import shutil

    
def detect_objects_on_images(zip_name, output_suffix, drive_path_root, known_classes=known_classes):

    path_to_zip = drive_path_root + zip_name
    path_extracted = "/content" + zip_name

    print('ZIP source:', path_to_zip)
    print("Extracting to", path_extracted)

    os.makedirs(path_extracted, exist_ok=True)

    with zipfile.ZipFile(path_to_zip, 'r') as zip_ref:
        zip_ref.extractall(path_extracted)

    image_paths = [path_extracted +'/'+p for p in os.listdir(path_extracted) if p.endswith(".png")]
    print(f'Found {len(image_paths)} images in {path_extracted}')

    detected_objects = []
    results_images_with_boxes = []

    for image_path in tqdm(image_paths):
        image = Image.open(image_path).convert("RGB")
        results = model.predict(image, verbose=False)

        # add detected objects with confidence to list
        for i in range(len(results)):
            for j in range(len(results[i].boxes)):
                predicted_class = known_classes[results[i].boxes[j].cls.int().item()]
                confidence = round(results[i].boxes[j].conf.item(), 2)

                detected_objects.append({
                    "image_path": image_path,
                    "predicted_class": predicted_class,
                    "confidence": confidence
                })

        # add image with boxes to list
        results_images_with_boxes.append({
            "image_path": image_path,
            "results": results[0]
        })

    output_detected_objects_name = f"detected_objects_{output_suffix}.csv"
    output_images_with_boxes_name = f"images_with_boxes_{output_suffix}"

    # save detected_objects as a csv
    df = pd.DataFrame(detected_objects)
    df.to_csv(output_detected_objects_name, index=True)

    # save images_with_boxes as a zip
    os.makedirs(output_images_with_boxes_name, exist_ok=True)
    for image in results_images_with_boxes:
        image['results'].save(output_images_with_boxes_name + '/' + image['image_path'].split('/')[-1])

    # zip the folder
    shutil.make_archive(output_images_with_boxes_name, 'zip', output_images_with_boxes_name)

    # # copy to google drive
    shutil.copyfile(output_detected_objects_name, drive_path_root + '/' + zip_name.split('/')[1] + '/' + output_detected_objects_name)
    print("Finished, saved to", drive_path_root + '/' + zip_name.split('/')[1] + '/' + output_detected_objects_name)

    shutil.copyfile(output_images_with_boxes_name + '.zip', drive_path_root + '/' + zip_name.split('/')[1] + '/' + output_images_with_boxes_name + '.zip')
    print("Finished, saved to", drive_path_root + '/' + zip_name.split('/')[1] + '/' + output_images_with_boxes_name + '.zip')
    
    # # clean up
    shutil.rmtree(output_images_with_boxes_name)
    shutil.rmtree(path_extracted)

In [24]:
root_path = "datasets_generated_ready_models"
zip_names_and_suffices = [
    ("/DALLE3/dalle_cars_zipped.zip", "cars"),
    ("/DALLE3/dalle_wildlife_zipped.zip", "wildlife"),

    ("/SD_XL/cars_zipped.zip", "cars"),
    ("/SD_XL/wildlife_zipped.zip", "wildlife"),
]

for zip_name, output_suffix in zip_names_and_suffices:
    detect_objects_on_images(zip_name, output_suffix, drive_path_root=root_path)


ZIP source: datasets_generated_ready_models/DALLE3/dalle_cars_zipped.zip
Extracting to /content/DALLE3/dalle_cars_zipped.zip
Found 50 images in /content/DALLE3/dalle_cars_zipped.zip


100%|██████████| 50/50 [00:43<00:00,  1.14it/s]


Finished, saved to datasets_generated_ready_models/DALLE3/detected_objects_cars.csv
Finished, saved to datasets_generated_ready_models/DALLE3/images_with_boxes_cars.zip
ZIP source: datasets_generated_ready_models/DALLE3/dalle_wildlife_zipped.zip
Extracting to /content/DALLE3/dalle_wildlife_zipped.zip
Found 50 images in /content/DALLE3/dalle_wildlife_zipped.zip


100%|██████████| 50/50 [00:14<00:00,  3.50it/s]


Finished, saved to datasets_generated_ready_models/DALLE3/detected_objects_wildlife.csv
Finished, saved to datasets_generated_ready_models/DALLE3/images_with_boxes_wildlife.zip
ZIP source: datasets_generated_ready_models/SD_XL/cars_zipped.zip
Extracting to /content/SD_XL/cars_zipped.zip
Found 50 images in /content/SD_XL/cars_zipped.zip


100%|██████████| 50/50 [00:14<00:00,  3.36it/s]


Finished, saved to datasets_generated_ready_models/SD_XL/detected_objects_cars.csv
Finished, saved to datasets_generated_ready_models/SD_XL/images_with_boxes_cars.zip
ZIP source: datasets_generated_ready_models/SD_XL/wildlife_zipped.zip
Extracting to /content/SD_XL/wildlife_zipped.zip
Found 50 images in /content/SD_XL/wildlife_zipped.zip


100%|██████████| 50/50 [00:13<00:00,  3.59it/s]


Finished, saved to datasets_generated_ready_models/SD_XL/detected_objects_wildlife.csv
Finished, saved to datasets_generated_ready_models/SD_XL/images_with_boxes_wildlife.zip
